This is my modified version of `run_hw1.ipynb`.

In [1]:
%cd C:\Users\DeadScholar\Programming\CS 285 Fall 2022\CS-285_fall2022\hw1

C:\Users\DeadScholar\Programming\CS 285 Fall 2022\CS-285_fall2022\hw1


In [2]:
#@title test GPU availability

import torch
torch.cuda.is_available()

True

In [3]:
#@title test virtual display

#@markdown If you see a video of a four-legged ant fumbling about, setup is complete!

import gym
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
)

env = wrap_env(gym.make("Ant-v4", render_mode='rgb_array'))

observation = env.reset()
for i in range(100):
    env.render()
    obs, rew, term, _ = env.step(env.action_space.sample() ) 
    if term:
      break;

env.close()
print('Loading video...')
show_video()

C:\Users\DeadScholar\Miniconda3\envs\cs285-t\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\DeadScholar\Miniconda3\envs\cs285-t\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\DeadScholar\Miniconda3\envs\cs285-t\lib\site-packages\gym\wrappers\record_video.py:78: UserWarning: WARN: Overwriting existing videos at C:\content\video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Loading video...


## Run Behavior Cloning (Problem 1)

In [4]:
#@title imports

import os
import time
import numpy as np
from ipywidgets import FloatProgress

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.bc_agent import BCAgent
from cs285.policies.loaded_gaussian_policy import LoadedGaussianPolicy
from cs285.infrastructure.utils import MJ_ENV_KWARGS, MJ_ENV_NAMES

%reload_ext autoreload
%load_ext autoreload
%autoreload 2

import logging
import ipdb
%pdb on

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Automatic pdb calling has been turned ON


C:\Users\DeadScholar\Miniconda3\envs\cs285-t\lib\site-packages\tensorboardX\proto\resource_handle_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\DeadScholar\Miniconda3\envs\cs285-t\lib\site-packages\tensorboardX\proto\resource_handle_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\DeadScholar\Miniconda3\envs\cs285-t\lib\site-packages\tensorboardX\proto\resource_handle_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or

Explanation of parameters

Experiment setup
* `expert_policy_file`: the relative location for the expert policy file (in `.pkl` format)
* `expert_data`: same, but for some data generated by the expert. The BC (behavior cloning) agent should emulate it. The DAgger agent emulates it at first, before using new expert-labelled data.
* `env_name`: the name of the gym environment. Can be Ant, Walker2d, HalfCheetah, Hopper.
* `exp_name`: the name of the experiment. Used by the tensorboard to tell the experiments apart.
* `do_dagger`: whether the experiment is going to use DAgger or BC.
* `save_params`: Whether the agent should dump its code somewhere for future reference.

* `ep_len`: Max-length per episode.

Network parameter:
* `n_layers`
* `size`

Optimizer parameter:
* `learning_rate`

Training parameter: 
* `num_agent_train_steps_per_iter`: number of `update` steps per iteration of training
* `train_batch_size`: number of sampled data points to be used per `update` step of the agent
* `batch_size`: number of env-interaction collected per iteration of training
* `max_replay_buffer_size`: max replay buffer size

Testing parameter:
* `n_iter`: number of iterations
* `eval_batch_size`: number of env-interaction collected for each iteration of evaluating

In [5]:
#@title runtime arguments

environment_name = "Hopper"
number_of_iterations = 1

episode_length = 10000
episodes_per_train_iteration = 1
env_interactions_per_train_iteration = episode_length * episodes_per_train_iteration
number_of_data_per_optimizer_step = 100

num_agent_train_steps_per_iter = 1000000 # change this

evals_per_iteration = 10
env_interactions_per_eval_iteration = episode_length * evals_per_iteration

number_of_hidden_layers_in_the_neural_network = 2
number_of_neurons_in_each_hidden_layer = 2**6

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  #@markdown expert data
  expert_policy_file = 'cs285/policies/experts/'+environment_name+'.pkl' #@param
  expert_data = 'cs285/expert_data/expert_data_'+environment_name+'-v4.pkl' #@param
  env_name = environment_name+'-v4' #@param ['Ant-v4', 'Walker2d-v4', 'HalfCheetah-v4', 'Hopper-v4']
  exp_name = 'bc_'+environment_name+'_'+str(num_agent_train_steps_per_iter)+'trainsteps' #@param
  do_dagger = False #@param {type: "boolean"}
  ep_len = episode_length #@param {type: "integer"}
  save_params = False #@param {type: "boolean"}

  num_agent_train_steps_per_iter = num_agent_train_steps_per_iter #@param {type: "integer"})
  n_iter = number_of_iterations #@param {type: "integer"})

  #@markdown batches & buffers
  batch_size = env_interactions_per_train_iteration #@param {type: "integer"})
  eval_batch_size = env_interactions_per_eval_iteration #@param {type: "integer"}
  train_batch_size = number_of_data_per_optimizer_step #@param {type: "integer"}
  max_replay_buffer_size = 1000000 #@param {type: "integer"}

  #@markdown network
  n_layers = number_of_hidden_layers_in_the_neural_network #@param {type: "integer"}
  size = number_of_neurons_in_each_hidden_layer #@param {type: "integer"}
  learning_rate = 5e-3 #@param {type: "number"}

  #@markdown logging
  video_log_freq = -1 #@param {type: "integer"}
  scalar_log_freq = 1 #@param {type: "integer"}

  #@markdown gpu & run-time settings
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

args = Args()

In [6]:
#@title create directory for logging

if args.do_dagger:
    logdir_prefix = 'q2_'  # The autograder uses the prefix `q2_`
    assert args.n_iter>1, ('DAgger needs more than 1 iteration (n_iter>1) of training, to iteratively query the expert and train (after 1st warmstarting from behavior cloning).')
else:
    logdir_prefix = 'q1_'  # The autograder uses the prefix `q1_`
    assert args.n_iter==1, ('Vanilla behavior cloning collects expert data just once (n_iter=1)')

data_path ='/content/cs285_f2022/hw1/data'
if not (os.path.exists(data_path)):
    os.makedirs(data_path)
logdir = logdir_prefix + args.exp_name + '_' + args.env_name + \
         '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

In [7]:
#@title define `BC_Trainer`
class BC_Trainer(object):

    def __init__(self, params):
        #######################
        ## AGENT PARAMS
        #######################

        agent_params = {
            'n_layers': params['n_layers'],
            'size': params['size'],
            'learning_rate': params['learning_rate'],
            'max_replay_buffer_size': params['max_replay_buffer_size'],
            }

        self.params = params
        self.params['agent_class'] = BCAgent
        self.params['agent_params'] = agent_params

        self.params["env_kwargs"] = MJ_ENV_KWARGS[self.params['env_name']]

        ################
        ## RL TRAINER
        ################

        self.rl_trainer = RL_Trainer(self.params)
        
        #######################
        ## LOAD EXPERT POLICY
        #######################

        print('Loading expert policy from...', self.params['expert_policy_file'])
        self.loaded_expert_policy = LoadedGaussianPolicy(self.params['expert_policy_file'])
        print('Done restoring expert policy...')

    def run_training_loop(self):

        self.rl_trainer.run_training_loop(
            n_iter=self.params['n_iter'],
            initial_expertdata=self.params['expert_data'],
            collect_policy=self.rl_trainer.agent.actor,
            eval_policy=self.rl_trainer.agent.actor,
            relabel_with_expert=self.params['do_dagger'],
            expert_policy=self.loaded_expert_policy,
        )


In [8]:
## run training
print(args.logdir)
trainer = BC_Trainer(args)
trainer.run_training_loop()

/content/cs285_f2022/hw1/data\q1_bc_Hopper_1000000trainsteps_Hopper-v4_13-09-2022_22-28-03
########################
logging outputs to  /content/cs285_f2022/hw1/data\q1_bc_Hopper_1000000trainsteps_Hopper-v4_13-09-2022_22-28-03
########################
Using GPU id 0
Loading expert policy from... cs285/policies/experts/Hopper.pkl
obs (1, 11) (1, 11)
Done restoring expert policy...


********** Iteration 0 ************

Training agent using sampled data from replay buffer...


KeyboardInterrupt: 

> c:\users\deadscholar\programming\cs 285 fall 2022\cs-285_fall2022\hw1\cs285\agents\bc_agent.py(37)sample()
     35 
     36     def sample(self, batch_size):
---> 37         return self.replay_buffer.sample_random_data(batch_size)  # HW1: you will modify this
     38 
     39     def save(self, path):



ipdb>  q


Run tensorboard with command `tensorboard --logdir /content/cs285_f2022/hw1/data` to see the experiment results. Alternatively you can do it right here in the notebook, but I would rather open it separately.

```python

%load_ext tensorboard
%tensorboard --logdir /content/cs285_f2022/hw1/data
```